In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, SimpleRNN, Masking, TimeDistributed, LSTM, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import numpy as np
import datetime as dt
import json
import os
import math
from matplotlib import pyplot as plt

Using TensorFlow backend.


In [1]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, SimpleRNN, Masking, TimeDistributed, LSTM, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import Sequence
from tensorflow.keras import backend as K
import numpy as np
import datetime as dt
import json
import os
import math
from matplotlib import pyplot as plt

#from tensorflow.contrib.opt import AdamWOptimizer # remove from tf 2.0

In [2]:
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
#tf.test.gpu_device_name()

In [32]:
DUMMY_VALUE = -1.0
DATA_PATH = "../TwitchHighlightCrawler/vod/" # const

TRAINING_TASK_ID = 'yb9ap3dscbr3p'
SPECIFIC_TRAINING_DATA_ID = ''
SPECIFIC_TRAINING_LABEL_ID = ''
SPECIFIC_GROUND_TRUTH_ID = ''
TESTING_TASK_ID = 'jf37cltm4hq68'
SPECIFIC_TESTING_DATA_ID = ''
SPECIFIC_TESTING_LABEL_ID = ''

WINDOW = 1
VIEWS_THRESHOLD = 3
CLIP_GRACE_PERIOD = 14

FEATURE_DATA_TYPE = np.float64

In [3]:
X = np.load('training_data-' + TRAINING_TASK_ID + (('-' + SPECIFIC_TRAINING_DATA_ID) if SPECIFIC_TRAINING_DATA_ID else '') + '.npy')
Y = np.load('training_label-' + TRAINING_TASK_ID + (('-' + SPECIFIC_TRAINING_LABEL_ID) if SPECIFIC_TRAINING_LABEL_ID else '') + '.npy')

In [5]:
def stringToDateTime(str): # only parse twitch info format
    str = (str[:19] + 'Z') if len(str) > 20 else str
    return dt.datetime.strptime(str, '%Y-%m-%dT%H:%M:%SZ')

def computeClassWeight(y): # the return value of this function already assume negative data is large than positive data
    positive = 0
    negative = 0
    
    for sample in y:
        for timestep in sample:
            if timestep[0] > 0:
                positive += 1
            elif not timestep[0]:
                negative += 1
            else:
                break
    
    return {
        0: 1,
        1: negative / positive
    }

def computeSampleWeight(y): # the return value of this function based on computeClassWeight()'s assumption 
    classWeight = computeClassWeight(y)
    
    sampleWeight = np.empty(y.shape[0:2])
    for sampleIndex, sample in enumerate(y):
        for timestepIndex, timestep in enumerate(sample):
            if timestep[0] > 0:
                sampleWeight[sampleIndex][timestepIndex] = classWeight[1]
            elif not timestep[0]:
                sampleWeight[sampleIndex][timestepIndex] = 1 # or classWeight[0]
            else:
                sampleWeight[sampleIndex][timestepIndex] = 0
                
    return sampleWeight

def K_precision(y_true, y_pred): # this function assume DUMMY_VALUE is -1.0
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        masking_size = K.sum(K.round(K.clip(y_pred * y_true, DUMMY_VALUE, 0))) # this will be a negative value (or -0)
        precision = true_positives / (predicted_positives + masking_size + K.epsilon()) # since masking will be a negative value, we plus it
        return precision

def K_recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def K_F1(y_true, y_pred):
    precision = K_precision(y_true, y_pred)
    recall = K_recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

dependencies = {
    'K_precision': K_precision,
    'K_recall': K_recall
}

In [5]:
w = computeSampleWeight(Y)

In [30]:
class dtatSequence(Sequence):
    def __init__(self, data_path, id_list, batch_size, shuffle = True):
        self.data_path = data_path
        self.id_list = id_list
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        if self.shuffle:
            np.random.shuffle(self.id_list)

    def __len__(self):
        return int(np.ceil(len(self.id_list) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_id = self.id_list[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        for data_id in batch_id:
            x = np.load()
        
        
        max_data_length = len(max(batch_x, key=len))
        feature_dimension = len(batch_x[0][0])
        
        # pad each data to the same length
        padding_x = np.full( (self.batch_size, max_data_length, feature_dimension), fill_value=DUMMY_VALUE, dtype=FEATURE_DATA_TYPE )
        padding_y = np.full( (self.batch_size, max_data_length, 1), fill_value=DUMMY_VALUE, dtype=int )
        for s, x in enumerate(batch_x):
            video_length = len(x)
            padding_x[s, 0:video_length, :] = x
        del batch_x
        for s, y in enumerate(batch_y):
            video_length = len(y)
            padding_y[s, 0:video_length, :] = y
        del batch_y

        return padding_x, padding_y
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.id_list)

In [7]:
class DataGenerator():
    def __init__(self, dim):
        self.kk = dim

aa = np.array([5,6,7])
a = DataGenerator(aa)
aa[0] = 4
print(a.kk) # [4 6 7]
a.kk[1] = 8
print(aa) # [4 8 7]

[4 6 7]
[4 8 7]


In [14]:
b = DataGenerator(list(range(5)))
print(b.kk)

[0, 1, 2, 3, 4]


In [6]:
model = Sequential()
model.add(Masking(mask_value=DUMMY_VALUE, input_shape=(None, 300)))
model.add(Bidirectional(LSTM(300, return_sequences=True, implementation=1)))
#model.add(LSTM(2, return_sequences=True))
#model.add(LSTM(1, return_sequences=True, activation='sigmoid'))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', K_precision, K_recall],
              sample_weight_mode='temporal')
model.summary()

modelName = 'SW-C2T2_WE_0401-0501_BILSTM300-tanh-i1-DENSE1-sigmoid_30-2'
checkpoint = ModelCheckpoint(filepath='./model_checkpoints/{epoch:02d}-{val_loss:.4f}_' + modelName + '.h5', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose=1)
histories = []
histories.append( model.fit(X, Y, epochs=30, batch_size=2, validation_split=0.1, callbacks=[es, checkpoint], sample_weight=w) ) #
model.save(modelName + '.h5')
#32/317 [==>...........................] - ETA: 52:08 - loss: 0.2944 - acc: 0.8699 - K_precision: 0.1527 - K_recall: 4.0953e-04
#32/317 [==>...........................] - ETA: 52:03 - loss: 5.4406 - acc: 0.8645 - K_precision: 0.0828 - K_recall: 0.0021

#1/317 [..............................] - ETA: 28:10:04 - loss: 0.4223 - acc: 0.1861 - K_precision: 0.1867 - K_recall: 0.9714
#1/317 [..............................] - ETA: 24:35:47 - loss: 4.9976 - acc: 0.7712 - K_precision: 0.0000e+00 - K_recall: 0.0000e+00

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 2)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, None, 4)           80        
_________________________________________________________________
time_distributed (TimeDistri (None, None, 1)           5         
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
Train on 317 samples, validate on 36 samples
Epoch 1/30
288/317 [==========================>...] - ETA: 3s - loss: 0.2956 - accuracy: 0.7848 - K_precision: 0.0884 - K_recall: 0.0659
Epoch 00001: val_loss improved from inf to 0.36979, saving model to ./model_checkpoints/01-0.3698_test.h5
317/317 [==============================] - 42s 132ms/sample - loss: 0.2921 - accuracy: 0.7838 - K_prec

KeyboardInterrupt: 

In [4]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=10))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 生成虚拟数据
import numpy as np
'''data = np.random.random((1000, 10))
labels = np.random.randint(2, size=(1000, 1))'''
data = []
labels = []
import random
for i in range(1000):
    data.append(np.random.random(10))
    labels.append(random.randint(0,1))

# 训练模型，以 32 个样本为一个 batch 进行迭代
model.fit(data, labels, epochs=10, batch_size=32)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 1000 arrays: [array([[0.51024308],
       [0.3404787 ],
       [0.07502523],
       [0.26591401],
       [0.2502586 ],
       [0.28769308],
       [0.43627578],
       [0.19952613],
       [0.87965262],
       [0....

In [70]:
'''data = np.array([])
data = np.append(data, np.random.random((10, 2)), axis=0)
#data = np.append(data, np.random.random((10, 2)), axis=0)
#np.concatenate((data, np.random.random((1,10, 2))),axis=0 )
np.random.random((10, 2))
data'''
import random
d = []
features = []
for i in range(10):
    feature = []
    for j in range(2):
        feature.append(random.random())
    features.append(feature)
    
d.append(features)
d.append(features)
np.array(d)

array([[[0.95156836, 0.5257626 ],
        [0.00156089, 0.85387701],
        [0.78281018, 0.73223741],
        [0.37595671, 0.49269121],
        [0.92361927, 0.67693445],
        [0.9214292 , 0.72112536],
        [0.66214929, 0.10112643],
        [0.90462236, 0.31870872],
        [0.15810725, 0.73876466],
        [0.53127291, 0.89857455]],

       [[0.95156836, 0.5257626 ],
        [0.00156089, 0.85387701],
        [0.78281018, 0.73223741],
        [0.37595671, 0.49269121],
        [0.92361927, 0.67693445],
        [0.9214292 , 0.72112536],
        [0.66214929, 0.10112643],
        [0.90462236, 0.31870872],
        [0.15810725, 0.73876466],
        [0.53127291, 0.89857455]]])

In [3]:
# 生成虚拟数据
import random

data = []
labels = []
max_seq_len = 0

N = 10 # number of data

dummy = -1
for i in range(N):
    length = random.randint(5,25)
    if length > max_seq_len:
        max_seq_len = length
    #length = 10
    data.append( np.random.random((length, 2))  )
    labels.append( np.random.randint(2, size=(length, 1)) )
#np.set_printoptions(threshold=False)

Xpad = np.full((N, max_seq_len, 2), fill_value=dummy, dtype='float64')
Ypad = np.full((N, max_seq_len, 1), fill_value=dummy, dtype='float64')
for s, x in enumerate(data):
    video_length = len(x)
    Xpad[s, 0:video_length, :] = x
for s, y in enumerate(labels):
    video_length = len(y)
    Ypad[s, 0:video_length, :] = y

#data = np.array(data)
#labels = np.array(labels)
#print(data.shape)
#print(labels.shape)



#input = Input(shape=(None, 2))

model = Sequential()
model.add(Masking(mask_value=dummy, input_shape=(None, 2)))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(1, return_sequences=True, activation='sigmoid'))
#model.add(Dense(1, activation='sigmoid'))
#model.add(TimeDistributed(Dense(2, activation='softmax')))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])


'''data = []
with np.printoptions(threshold=np.inf):
    print(Xpad)
    print(data)'''
# 训练模型，以 32 个样本为一个 batch 进行迭代
#model.fit(data, labels, epochs=10, batch_size=8)
model.fit(Xpad, Ypad, epochs=10, batch_size=8)

W0821 19:51:54.256520  4772 deprecation_wrapper.py:119] From D:\Users\User\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0821 19:51:54.494101  4772 deprecation_wrapper.py:119] From D:\Users\User\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0821 19:51:54.571093  4772 deprecation_wrapper.py:119] From D:\Users\User\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0821 19:51:54.920798  4772 deprecation.py:323] From D:\Users\User\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\backend\tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorfl

Epoch 1/10
10/10 [==============================] - 10s 989ms/step - loss: 0.7155 - acc: 0.5360
Epoch 2/10
10/10 [==============================] - 0s 6ms/step - loss: 0.6940 - acc: 0.5642
Epoch 3/10
10/10 [==============================] - 0s 6ms/step - loss: 0.6936 - acc: 0.5517
Epoch 4/10
10/10 [==============================] - 0s 5ms/step - loss: 0.6891 - acc: 0.5585
Epoch 5/10
10/10 [==============================] - 0s 6ms/step - loss: 0.6891 - acc: 0.5601
Epoch 6/10
10/10 [==============================] - 0s 6ms/step - loss: 0.6850 - acc: 0.5660
Epoch 7/10
10/10 [==============================] - 0s 6ms/step - loss: 0.6834 - acc: 0.5632
Epoch 8/10
10/10 [==============================] - 0s 5ms/step - loss: 0.6814 - acc: 0.5998
Epoch 9/10
10/10 [==============================] - 0s 5ms/step - loss: 0.6785 - acc: 0.5844
Epoch 10/10
10/10 [==============================] - 0s 6ms/step - loss: 0.6771 - acc: 0.5918


In [8]:
test = []

'''import random
features = []
for i in range(10):
    feature = []
    for j in range(2):
        feature.append(random.random())
    features.append(feature)
test.append(features)
test1 = np.array(test)

test = []
features = []
for i in range(6):
    feature = []
    for j in range(2):
        feature.append(random.random())
    features.append(feature)
test.append(features)
test2 = np.array(test)

test = np.array(test)
with np.printoptions(threshold=np.inf):
    for i in range(2):
        print(model.predict(test, batch_size=8))'''

'''test = np.array([ [[[2,5],[3,6],[4,8]]], [[[2,5],[3,6]]] ]) # 4維 沒用
with np.printoptions(threshold=np.inf):
    print(model.predict(test[0], batch_size=8))'''

'''a = np.array([[[2,5],[3,6],[4,8]]])
b = np.array([[[2,5],[3,6]]])
c = []
c.append(a)
c.append(b)
test = np.array(c) # 無解 這行出錯
with np.printoptions(threshold=np.inf):
    print(model.predict(test[0], batch_size=8))'''

'''test.append(np.random.random((10, 2)))
test.append(np.random.random((6, 2)))
#test = np.array(test)
Tpad = np.full((2, 10, 2), fill_value=dummy, dtype='Float64')
for s, t in enumerate(test):
    video_length = len(t)
    Tpad[s, 0:video_length, :] = t

with np.printoptions(threshold=np.inf):
    print(model.predict(Tpad, batch_size=8))'''

[array([[[2, 5],
        [3, 6],
        [4, 8]]]), array([[[2, 5],
        [3, 6]]])]


ValueError: could not broadcast input array from shape (3,2) into shape (1)

In [8]:
adamw = AdamWOptimizer(weight_decay=0.01)

In [22]:
model = Sequential()
model.add(Masking(mask_value=DUMMY_VALUE, input_shape=(None, 2)))
model.add(SimpleRNN(1, return_sequences=True, activation='sigmoid'))
model.compile(optimizer=adamw,
              loss='binary_crossentropy',
              metrics=['accuracy', K_F1, K_precision, K_recall],
             sample_weight_mode='temporal')

X = np.array([[[1, 2], [2, 3], [3, 4]], [[4, 5], [6, 9], [-1, -1]]])
Y = np.array([[[0], [1], [0]], [[0], [0], [-1]]])
#w = np.array([[1,1,1], [1,1,0]])
w = computeSampleWeight(Y)

model.fit(X, Y, epochs=10, batch_size=8, sample_weight=w)
#model.predict(X)
prediction = model.predict_classes(X)
prediction
#type(prediction[0][0][0]) # numpy.int32

Epoch 1/10
2/2 [==============================] - 1s 267ms/sample - loss: 1.3977 - acc: 0.2000 - K_F1: 0.3333 - K_precision: 0.2000 - K_recall: 1.0000
Epoch 2/10
2/2 [==============================] - 0s 4ms/sample - loss: 1.3896 - acc: 0.2000 - K_F1: 0.3333 - K_precision: 0.2000 - K_recall: 1.0000
Epoch 3/10
2/2 [==============================] - 0s 3ms/sample - loss: 1.3808 - acc: 0.2000 - K_F1: 0.3333 - K_precision: 0.2000 - K_recall: 1.0000
Epoch 4/10
2/2 [==============================] - 0s 3ms/sample - loss: 1.3715 - acc: 0.2000 - K_F1: 0.3333 - K_precision: 0.2000 - K_recall: 1.0000
Epoch 5/10
2/2 [==============================] - 0s 2ms/sample - loss: 1.3621 - acc: 0.2000 - K_F1: 0.3333 - K_precision: 0.2000 - K_recall: 1.0000
Epoch 6/10
2/2 [==============================] - 0s 2ms/sample - loss: 1.3525 - acc: 0.2000 - K_F1: 0.3333 - K_precision: 0.2000 - K_recall: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 3ms/sample - loss: 1.3429 - acc: 0.2000 - K_F1: 0.3

array([[[1],
        [1],
        [1]],

       [[1],
        [1],
        [1]]])

In [16]:
a = np.array([0.48, 0.5, 0.51])
b = np.array([0, -1, -1])
np.round(np.clip(a*b, -1, 0))

array([ 0., -0., -1.])

In [27]:
import random

data = []
labels = []
max_seq_len = 0

N = 10 # number of data

dummy = -1
for i in range(N):
    length = random.randint(5,25)
    if length > max_seq_len:
        max_seq_len = length
    #length = 10
    data.append( np.random.random((length, 2))  )
    labels.append( np.random.randint(2, size=(length, 1)) )

In [35]:
model = Sequential()
model.add(Masking(mask_value=DUMMY_VALUE, input_shape=(None, 2)))
model.add(SimpleRNN(1, return_sequences=True, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', K_F1, K_precision, K_recall],
             sample_weight_mode='temporal')

X = np.array(data)
Y = np.array(labels)

model.fit_generator(dtatSequence(X, Y, 8), epochs=10)
X = np.array([[[1, 2], [2, 3], [3, 4]], [[4, 5], [6, 9], [-1, -1]]])



Epoch 1/10
2/2 [==============================] - 1s 514ms/step - loss: 0.3068 - accuracy: 0.5280 - K_F1: 0.6536 - K_precision: 0.5082 - K_recall: 0.9162
Epoch 2/10
2/2 [==============================] - 0s 53ms/step - loss: 0.3066 - accuracy: 0.5280 - K_F1: 0.6536 - K_precision: 0.5082 - K_recall: 0.9162
Epoch 3/10
2/2 [==============================] - 0s 47ms/step - loss: 0.3062 - accuracy: 0.5280 - K_F1: 0.6536 - K_precision: 0.5082 - K_recall: 0.9162
Epoch 4/10
2/2 [==============================] - 0s 49ms/step - loss: 0.3060 - accuracy: 0.5280 - K_F1: 0.6536 - K_precision: 0.5082 - K_recall: 0.9162
Epoch 5/10
2/2 [==============================] - 0s 49ms/step - loss: 0.3057 - accuracy: 0.5280 - K_F1: 0.6536 - K_precision: 0.5082 - K_recall: 0.9162
Epoch 6/10
2/2 [==============================] - 0s 64ms/step - loss: 0.3054 - accuracy: 0.5280 - K_F1: 0.6536 - K_precision: 0.5082 - K_recall: 0.9162
Epoch 7/10
2/2 [==============================] - 0s 37ms/step - loss: 0.3050 - a

array([[[0],
        [0],
        [0]],

       [[0],
        [0],
        [0]]])

In [3]:
def local_score(clipDataList, predict, numberOfPredictedFragment):
    localPrecision = 0
    localRecall = 0
    
    videoLength = len(predict)
    
    for clip in clipDataList:
        recall = 0
        overlapTime = 0 # for local recall computing

        endOfClip = clip['offset'] + clip['duration'] # not real video end timeline point

        for i in range(clip['offset'] - (MAX_CLIP_LENGTH-1), clip['offset'] - MIN_CLIP_LENGTH + 1): # (offset - 59) <= prediction <= (offset - 5) # eg. offset = 85, duration = 10, 26~80
            if predict[i] and (i + predict[i]['duration']) > clip['offset']: # Simplify from (i + (predict[i]['duration']-1)) >= clip['offset']
                endOfPrediction = i + predict[i]['duration']
                overlap = endOfPrediction - clip['offset'] # compute overlap seconds

                if endOfPrediction > endOfClip:
                    overlap -= (endOfPrediction - endOfClip) # remove the part over than the end of ground true

                # precision
                predict[i]['precision'].append( overlap/predict[i]['duration'] )
                # local recall
                singleRecallValue = overlap / clip['duration']
                recall += singleRecallValue
                overlapTime += 1

        for i in range(clip['offset'] - MIN_CLIP_LENGTH + 1, clip['offset']): # (offset - 4(less than MIN_CLIP_LENGTH)) <= prediction < offset # 81~84
            if predict[i]: # no need to check if it overlap the ground truth period, it must be in the period
                endOfPrediction = i + predict[i]['duration']
                overlap = endOfPrediction - clip['offset']; # compute overlap seconds

                if endOfPrediction > endOfClip: # predicted fragment is larger than clip
                    overlap -= (endOfPrediction - endOfClip); # remove the part over than the end of ground true

                # precision
                predict[i]['precision'].append( overlap/predict[i]['duration'] )
                # local recall
                singleRecallValue = overlap / clip['duration']
                recall += singleRecallValue
                overlapTime += 1

        for i in range(clip['offset'], endOfClip): # offset <= prediction < (offset + duration) # 85~94
            if predict[i]: # no need to check if it overlap the ground truth period, it must be in the period
                endOfPrediction = i + predict[i]['duration']
                overlap = endOfClip - i; # compute overlap seconds

                if endOfPrediction < endOfClip: # predicted fragment is smaller than clip
                    overlap -= endOfClip - endOfPrediction

                # precision
                predict[i]['precision'].append( overlap/predict[i]['duration'] )
                # local recall
                singleRecallValue = overlap / clip['duration']
                recall += singleRecallValue
                overlapTime += 1

        if overlapTime:
            recall /= overlapTime
            localRecall += recall
    
    for i in range(videoLength):
        if predict[i]: # check have predicted fragment
            if len(predict[i]['precision']): # the fragment has overlap
                precision = 0 # for local precision computing

                for singlePrecisionValue in predict[i]['precision']:
                    # local precision
                    precision += singlePrecisionValue
                precision /= len(predict[i]['precision'])

                localPrecision += precision

    localPrecision /= numberOfPredictedFragment
    localRecall /= len(clipDataList)
    F1 = (2 * localPrecision * localRecall) / (localPrecision + localRecall)
    print(str(localPrecision) + ' ' + str(localRecall) + ' ' + str(F1))
    
    return {
        'precision': precision,
        'recall': recall,
        'F1': F1
    }
    
def macro_score(clipDataList, predict):
    macroPrecision = 0
    macroRecall = 0
    macroF1 = 0
    
    videoLength = len(predict)
    
    for clip in clipDataList:
        precision = 0
        recall = 0
        F1 = 0
        
        overlapTime = 0

        endOfClip = clip['offset'] + clip['duration'] # not real video end timeline point

        for i in range(clip['offset'] - (MAX_CLIP_LENGTH-1), clip['offset'] - MIN_CLIP_LENGTH + 1): # (offset - 59) <= prediction <= (offset - 5) # eg. offset = 85, duration = 10, 26~80
            if predict[i] and (i + predict[i]['duration']) > clip['offset']: # Simplify from (i + (predict[i]['duration']-1)) >= clip['offset']
                endOfPrediction = i + predict[i]['duration']
                overlap = endOfPrediction - clip['offset'] # compute overlap seconds

                if endOfPrediction > endOfClip:
                    overlap -= (endOfPrediction - endOfClip) # remove the part over than the end of ground true

                # precision
                singlePrecisionVlaue = overlap / predict[i]['duration']
                precision += singlePrecisionVlaue
                # recall
                singleRecallValue = overlap / clip['duration']
                recall += singleRecallValue
                # F1
                singleF1Value = (2 * singlePrecisionVlaue * singleRecallValue) / (singlePrecisionVlaue + singleRecallValue)
                F1 += singleF1Value
                
                overlapTime += 1

        for i in range(clip['offset'] - MIN_CLIP_LENGTH + 1, clip['offset']): # (offset - 4(less than MIN_CLIP_LENGTH)) <= prediction < offset # 81~84
            if predict[i]: # no need to check if it overlap the ground truth period, it must be in the period
                endOfPrediction = i + predict[i]['duration']
                overlap = endOfPrediction - clip['offset']; # compute overlap seconds

                if endOfPrediction > endOfClip: # predicted fragment is larger than clip
                    overlap -= (endOfPrediction - endOfClip); # remove the part over than the end of ground true

                # precision
                singlePrecisionVlaue = overlap / predict[i]['duration']
                precision += singlePrecisionVlaue
                # recall
                singleRecallValue = overlap / clip['duration']
                recall += singleRecallValue
                # F1
                singleF1Value = (2 * singlePrecisionVlaue * singleRecallValue) / (singlePrecisionVlaue + singleRecallValue)
                F1 += singleF1Value
                
                overlapTime += 1

        for i in range(clip['offset'], endOfClip): # offset <= prediction < (offset + duration) # 85~94
            if predict[i]: # no need to check if it overlap the ground truth period, it must be in the period
                endOfPrediction = i + predict[i]['duration']
                overlap = endOfClip - i; # compute overlap seconds

                if endOfPrediction < endOfClip: # predicted fragment is smaller than clip
                    overlap -= endOfClip - endOfPrediction

                # precision
                singlePrecisionVlaue = overlap / predict[i]['duration']
                precision += singlePrecisionVlaue
                # recall
                singleRecallValue = overlap / clip['duration']
                recall += singleRecallValue
                # F1
                singleF1Value = (2 * singlePrecisionVlaue * singleRecallValue) / (singlePrecisionVlaue + singleRecallValue)
                F1 += singleF1Value
                
                overlapTime += 1

        if overlapTime:
            precision /= overlapTime
            recall /= overlapTime
            F1 /= overlapTime
            
            macroPrecision += precision
            macroRecall += recall
            macroF1 += F1
    
    numberOfAnswer = len(clipDataList)
    
    macroPrecision /= numberOfAnswer
    macroRecall /= numberOfAnswer
    macroF1 /= numberOfAnswer  
    print(str(macroPrecision) + ' ' + str(macroRecall) + ' ' + str(macroF1))
    
def K_simple_global_score(label, prediction): # use to evaluation training data
    THRESHOLD = 0.5
    
    overlap = 0
    totalClipSeconds = 0
    totalPredictionSeconds = 0
    
    length = len(label)
    for i in range(length):
        if label[i][0] == -1:
            break
            
        if prediction[i][0] > THRESHOLD:
            totalPredictionSeconds += 1
            if label[i][0]:
                totalClipSeconds += 1
                overlap += 1
        elif label[i][0]:
            totalClipSeconds += 1
            
    precision = (overlap / totalPredictionSeconds) if totalPredictionSeconds > 0 else 0
    recall = (overlap / totalClipSeconds) if totalClipSeconds > 0 else 0
    F1 = ((2 * precision * recall) / (precision + recall)) if (precision + recall) > 0 else 0
            
    return {
        'precision': precision,
        'recall': recall,
        'F1': F1
    }

def global_score(groundTruth, predict):
    videoLength = len(groundTruth)
    
    # normal score
    overlap = 0
    totalClipSeconds = 0 # WARNING!! we can count this value when computing local score
    totalPredictionSeconds = 0
    
    # weighted score
    weightedOverlap = 0
    totalWeightedClipSeconds = 0 # WARNING!! we can count this value when computing local score
    totalWeightedPredictionSeconds = 0
    
    for i in range(videoLength):
        if groundTruth[i]:
            totalClipSeconds += 1
            totalWeightedClipSeconds += groundTruth[i]
            
            if predict[i]:
                totalPredictionSeconds += 1
                totalWeightedPredictionSeconds += groundTruth[i]
                
                overlap += 1
                weightedOverlap += groundTruth[i]
        elif predict[i]:
            totalPredictionSeconds += 1
            totalWeightedPredictionSeconds += 1 # no clip on this second, so weight is 1
                
    precision = overlap / totalPredictionSeconds if totalPredictionSeconds > 0 else 0
    recall = overlap / totalClipSeconds if totalClipSeconds > 0 else 0
    F1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    weightedPrecision = weightedOverlap / totalWeightedPredictionSeconds if totalWeightedPredictionSeconds > 0 else 0
    weightedRecall = weightedOverlap / totalWeightedClipSeconds if totalWeightedClipSeconds > 0 else 0
    weightedF1 = (2 * weightedPrecision * weightedRecall) / (weightedPrecision + weightedRecall) if (weightedPrecision + weightedRecall) > 0 else 0
    
    #print(str(precision) + ' ' + str(recall) + ' ' + str(F1))
    #print(str(weightedPrecision) + ' ' + str(weightedRecall) + ' ' + str(weightedF1))
    
    return {
        'normal': {
            'precision': precision,
            'recall': recall,
            'F1': F1
        },
        'weighted': {
            'precision': weightedPrecision,
            'recall': weightedRecall,
            'F1': weightedF1
        }
    }

def K_global_score(groundTruth, prediction):
    THRESHOLD = 0.5
    videoLength = len(groundTruth)
    
    # normal score
    overlap = 0
    totalClipSeconds = 0
    totalPredictionSeconds = 0
    
    # weighted score
    weightedOverlap = 0
    totalWeightedClipSeconds = 0
    totalWeightedPredictionSeconds = 0
    
    for i in range(videoLength):
        if groundTruth[i]:
            totalClipSeconds += 1
            totalWeightedClipSeconds += groundTruth[i]
            
            if prediction[i][0] > THRESHOLD:
                totalPredictionSeconds += 1
                totalWeightedPredictionSeconds += groundTruth[i]
                
                overlap += 1
                weightedOverlap += groundTruth[i]
        elif prediction[i][0] > THRESHOLD:
            totalPredictionSeconds += 1
            totalWeightedPredictionSeconds += 1 # no clip on this second, so weight is 1
                
    precision = overlap / totalPredictionSeconds if totalPredictionSeconds > 0 else 0
    recall = overlap / totalClipSeconds if totalClipSeconds > 0 else 0
    F1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    weightedPrecision = weightedOverlap / totalWeightedPredictionSeconds if totalWeightedPredictionSeconds > 0 else 0
    weightedRecall = weightedOverlap / totalWeightedClipSeconds if totalWeightedClipSeconds > 0 else 0
    weightedF1 = (2 * weightedPrecision * weightedRecall) / (weightedPrecision + weightedRecall) if (weightedPrecision + weightedRecall) > 0 else 0
    
    #print(str(precision) + ' ' + str(recall) + ' ' + str(F1))
    #print(str(weightedPrecision) + ' ' + str(weightedRecall) + ' ' + str(weightedF1))
    print(str(precision) + ' ' + str(recall) + ' ' + str(F1) + ' ' + str(weightedPrecision) + ' ' + str(weightedRecall) + ' ' + str(weightedF1))
    
    return {
        'normal': {
            'precision': precision,
            'recall': recall,
            'F1': F1
        },
        'weighted': {
            'precision': weightedPrecision,
            'recall': weightedRecall,
            'F1': weightedF1
        }
    }

def evaluation(channel, video, predict): # WARNING!! it hasn't deal that answer is out of the video yet
    global_answer = np.zeros(len(predict['macro']), dtype=int) # WARNING!! use dependent data len(predict['macro'])
    
    clipList = os.listdir(DATA_PATH + channel + '/'+ video +'/clip')
    clipDataList = [] # ground true for loacl & macro
    for clip in clipList:
        with open(DATA_PATH + channel + '/'+ video +'/clip/' + clip, "r", encoding="utf-8") as file:
            data = file.read()

        data = json.loads(data)
        
        # generate ground true for local & macro evaluation metrics
        clipDataList.append({
            'offset': data['vod']['offset'],
            'duration': math.ceil( data['duration'] ) # length of ground true clip
        })
        
        # generate ground true for global evaluation metrics
        for i in range(clipDataList[-1]['offset'], clipDataList[-1]['offset'] + clipDataList[-1]['duration']):
            global_answer[i] += 1
    
    print(predict['count'])
    local_score(clipDataList, predict['macro'], predict['count'])
    #macro_score(clipDataList, predict['macro'])
    global_score(global_answer, predict['global'])
    
def K_evaluationGlobalOnly(channel, video, prediction, clip_deadline = dt.datetime(9999,12,31,23,59,59), filter_low_views = False):
    VIDEO_LENGTH = len(prediction) # WARNING!! use dependent data len(prediction)
    
    global_answer = np.zeros(VIDEO_LENGTH, dtype=int)
    
    CLIP_PATH = DATA_PATH + channel + '/'+ video +'/clip/'
    clipList = os.listdir(CLIP_PATH)
    for clip in clipList:
        with open(CLIP_PATH + clip, "r", encoding="utf-8") as file:
            data = file.read()
        clipInfo = json.loads(data)
        
        if filter_low_views and clipInfo['views'] < VIEWS_THRESHOLD:
            continue # filter out a clip with low views
        if stringToDateTime(clipInfo['created_at']) >= clip_deadline:
            continue # filter out a clip created outside the range
        
        offset = clipInfo['vod']['offset']
        duration = math.ceil( clipInfo['duration'] ) # length of ground true clip

        if offset >= VIDEO_LENGTH: # check a clip is locate in the video range
            continue

        # generate ground true for global evaluation metrics
        for i in range(offset, offset + duration):
            if i >= VIDEO_LENGTH:
                break
            global_answer[i] += 1
    
    #print(channel + ' ' + video)
    return K_global_score(global_answer, prediction)

In [ ]:
TRAINING_MARK = np.load('training_mark-' + TRAINING_TASK_ID + '.npy')

# evaluate training set
precision = 0
recall = 0
F1 = 0

weightedPrecision = 0
weightedRecall = 0
weightedF1 = 0

for mark, prediction in zip(TRAINING_MARK, predictions):
    with open(DATA_PATH + mark[0] + '/' + mark[1] + '/info.json', "r", encoding="utf-8") as file: # mark[0]: channel, mark[1]: video
        data = file.read()
    videoInfo = json.loads(data)
        
    if WINDOW > 1:
        realPrediction = np.empty( (videoInfo['length'], 1), dtype=int )
        
        predictionLength = len(prediction)
        for t in range(predictionLength):
            realVideoIndex = t * WINDOW
            upperBound = realVideoIndex + WINDOW # end of this window 
            
            if upperBound > videoInfo['length']: # check if is out of the range
                upperBound = videoInfo['length']
            realPrediction[realVideoIndex:upperBound] = prediction[t]
    else:
        realPrediction = prediction[:videoInfo['length']]
        
    clipDeadline = stringToDateTime(videoInfo['created_at']) + dt.timedelta(seconds=videoInfo['length']) + dt.timedelta(days=CLIP_GRACE_PERIOD)

    scoreData = K_evaluationGlobalOnly(mark[0], mark[1], realPrediction, clipDeadline, True)
    precision += scoreData['normal']['precision']
    recall += scoreData['normal']['recall']
    F1 += scoreData['normal']['F1']
    weightedPrecision += scoreData['weighted']['precision']
    weightedRecall += scoreData['weighted']['recall']
    weightedF1 += scoreData['weighted']['F1']
    
precision /= len(predictions)
recall /= len(predictions)
F1 /= len(predictions)
weightedPrecision /= len(predictions)
weightedRecall /= len(predictions)
weightedF1 /= len(predictions)
print(str(precision) + ' ' + str(recall) + ' ' + str(F1))
print(str(weightedPrecision) + ' ' + str(weightedRecall) + ' ' + str(weightedF1))

In [ ]:
GROUND_TRUTH = np.load('ground_truth-' + TRAINING_TASK_ID + (('-' + SPECIFIC_GROUND_TRUTH_ID) if SPECIFIC_GROUND_TRUTH_ID else '') +'.npy', allow_pickle=True)

# evaluate training set (if has ground truth data)
precision = 0
recall = 0
F1 = 0

weightedPrecision = 0
weightedRecall = 0
weightedF1 = 0

for prediction, label in zip(predictions, GROUND_TRUTH):
    videoLength = len(label) # there is no padding in label data
    
    if WINDOW > 1:
        realPrediction = np.empty( (videoLength, 1), dtype=int )
        
        predictionLength = len(prediction)
        for t in range(predictionLength):
            realVideoIndex = t * WINDOW
            upperBound = realVideoIndex + WINDOW # end of this window 
            
            if upperBound > videoLength: # check if is out of the range
                upperBound = videoLength
            realPrediction[realVideoIndex:upperBound] = prediction[t]
    else:
        realPrediction = prediction[:videoLength]
    
    scoreData = K_global_score(label, realPrediction)
    precision += scoreData['normal']['precision']
    recall += scoreData['normal']['recall']
    F1 += scoreData['normal']['F1']
    weightedPrecision += scoreData['weighted']['precision']
    weightedRecall += scoreData['weighted']['recall']
    weightedF1 += scoreData['weighted']['F1']
    
precision /= len(predictions)
recall /= len(predictions)
F1 /= len(predictions)
weightedPrecision /= len(predictions)
weightedRecall /= len(predictions)
weightedF1 /= len(predictions)
print(str(precision) + ' ' + str(recall) + ' ' + str(F1))
print(str(weightedPrecision) + ' ' + str(weightedRecall) + ' ' + str(weightedF1))

In [ ]:
# evaluate testing set
precision = 0
recall = 0
F1 = 0

weightedPrecision = 0
weightedRecall = 0
weightedF1 = 0

for prediction, label in zip(predictions, TESTING_LABEL):
    videoLength = len(label) # there is no padding in label data
    
    if WINDOW > 1:
        realPrediction = np.empty( (videoLength, 1), dtype=int )
        
        predictionLength = len(prediction)
        for t in range(predictionLength):
            realVideoIndex = t * WINDOW
            upperBound = realVideoIndex + WINDOW # end of this window 
            
            if upperBound > videoLength: # check if is out of the range
                upperBound = videoLength
            realPrediction[realVideoIndex:upperBound] = prediction[t]
    else:
        realPrediction = prediction[:videoLength]
    
    scoreData = K_global_score(label, realPrediction)
    precision += scoreData['normal']['precision']
    recall += scoreData['normal']['recall']
    F1 += scoreData['normal']['F1']
    weightedPrecision += scoreData['weighted']['precision']
    weightedRecall += scoreData['weighted']['recall']
    weightedF1 += scoreData['weighted']['F1']
    
precision /= len(predictions)
recall /= len(predictions)
F1 /= len(predictions)
weightedPrecision /= len(predictions)
weightedRecall /= len(predictions)
weightedF1 /= len(predictions)
print(str(precision) + ' ' + str(recall) + ' ' + str(F1))
print(str(weightedPrecision) + ' ' + str(weightedRecall) + ' ' + str(weightedF1))

In [ ]:
def evaluateBaseline(mark, window=1):
    CLIP_GRACE_PERIOD = 14
    
    # score about frequency method
    precisionForFrequencyMethod = 0
    recallForFrequencyMethod = 0
    F1ForFrequencyMethod = 0

    weightedPrecisionForFrequencyMethod = 0
    weightedRecallForFrequencyMethod = 0
    weightedF1ForFrequencyMethod = 0
    
    # score about diversity method
    precisionForDiversityMethod = 0
    recallForDiversityMethod = 0
    F1ForDiversityMethod = 0

    weightedPrecisionForDiversityMethod = 0
    weightedRecallForDiversityMethod = 0
    weightedF1ForDiversityMethod = 0
    
    for videInfo in mark:
        VIDEO_PATH = DATA_PATH + mark[0] + '/'+ mark[1] +'/' # local const
        
        # read video info
        with open(VIDEO_PATH + 'info.json', "r", encoding="utf-8") as file:
            data = file.read()
        videoInfo = json.loads(data)
        
        # process message data (it can be optimized)
        messages = [[] for i in range(videoInfo['length'])]

        messagePathList = glob.glob(VIDEO_PATH + 'Message-*.json')
        for path in messagePathList:
            with open(path, "r", encoding="utf-8") as file:
                data = file.read()

            commentData = json.loads(data)['comments']
            for comment in commentData:
                offset = math.floor( comment['content_offset_seconds'] ) # get comment offset

                if offset >= videoInfo['length']:
                    break
                if only_chat and comment['source'] != 'chat':
                    if comment['source'] != 'comment':
                        print(channel + ' ' + video + ' ' + str(offset) + ' ' + comment['source'])
                    continue
                if comment['state'] != 'published':
                    print(channel + ' ' + video + ' ' + str(offset) + ' ' + comment['state'])

                messages[offset].append( comment['message']['body'] )
                
        

In [19]:
a=np.zeros((5,1))
b=np.array([[1],[0],[5],[1],[0]])
a[2:3] = b[2]
a

array([[0.],
       [0.],
       [5.],
       [0.],
       [0.]])

In [30]:
if(b[-1]):
    print(87)
else:
    print(899)

899


In [14]:
a = np.array([[1,2,3], [1,2], [1,2,3,4,5], [1,2,3,4]])
len(max(a, key=len))

5

In [17]:
a = [1,2,3,4,5,6,7]
b = np.full( 10, fill_value=DUMMY_VALUE, dtype=int )
b[:len(a)] = a
del a
b

array([ 1,  2,  3,  4,  5,  6,  7, -1, -1, -1])